In [1]:
import numpy as np
import pandas as pd
import fasttext
import os
import os.path
from sklearn.metrics import precision_score, recall_score, f1_score 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

In [2]:
dirs = os.getcwd()
len(os.listdir(dirs))

122

In [3]:
'''
 *各种字符的unicode编码的范围：
     * 汉字：[0x4e00,0x9fa5]（或十进制[19968,40869]）
     * 数字：[0x30,0x39]（或十进制[48, 57]）
     *小写字母：[0x61,0x7a]（或十进制[97, 122]）
     * 大写字母：[0x41,0x5a]（或十进制[65, 90]）
'''
import re
zhPattern = re.compile(u'[\u4e00-\u9fa5]+')
numPattern = re.compile('[0-9]')
def deal(x):
    if x == -888:  #这是缺失值填充的（记得先填充缺失值）
        return x
    contents = str(x)
    zhmatch = zhPattern.search(contents)
    if zhmatch:  
#         print (u'有中文：%s' % (zhmatch.group(0),))  #记住是怎么打印的
        return -666  #汉字 直接返回
    numbermatch = numPattern.search(contents)
    if numbermatch:
        return 8 #表示 数字 
    else:
#         print (u'有字母：%s' % (x))  #记住是怎么打印的
        return 9 #表示字母
        

In [4]:
def prepross(data):
    #1 使用填充的方式把
    dataPro = data.copy()
    label = dataPro.columns
    #统计每行的缺失率
    dataPro.insert(0,"missingRate",0)
    dataPro["missingRate"] = dataPro.isnull().sum(axis = 1)/dataPro.shape[1]
    
    dataPro.fillna(-888,inplace = True)
    # 数字和空缺值填充，保留分布格局
    for i in range(len(label)):
        dataPro[label[i]] = dataPro[label[i]].apply(lambda x: deal(x) )
        
    dataPro.insert(1,"index",dataPro.index)
    dataPro.insert(2,"position",0)
    dataPro.insert(3,"first_last_No",0)
    dataPro.insert(4,"class2_number",0)
    dataPro.insert(5,"alphabet",0)
    
    number1 = 0
    for i in range(dataPro.shape[0]):
        if i < int(dataPro.shape[0]/3):
            dataPro["position"][i] = 1
        elif i < int(2*dataPro.shape[0]/3) and i >= int(dataPro.shape[0]/3):
            dataPro["position"][i] = 2
        else:
            dataPro["position"][i] = 3

        # 下面表示的是 头部加上序列号（2标签可能全是数字），尾部全部用 -1
        #  (dataPro.iloc[i,:] == -666).sum() 比较保险，data[i] 出错。
        if (dataPro.iloc[i,:] == 8).sum() < (dataPro.iloc[i,:] == -666).sum() and number1 == 0:
                dataPro["first_last_No"][i] = i
        else:
             #针对 class2 全是数字的情况
            if i < int(dataPro.shape[0]/3):
                if (dataPro.iloc[i+1,:] == 8).sum() < (dataPro.iloc[i+1,:] == -666).sum()and number1 == 0: 
                    dataPro["first_last_No"][i] = i  #仍然编码 ，原来是i
                    dataPro["class2_number"][i] = 1 
                else:    
                    number1 += 1
        #中间class3 不用管 就是0
        if (dataPro.iloc[i,:] == 8).sum() < (dataPro.iloc[i,:] == -666).sum() and number1 > 0:
               dataPro["first_last_No"][i] = -1 
        # 对字母比较多的行进行标记
        if (dataPro.iloc[i,:] == 9).sum() > (dataPro.iloc[i,:] == -666).sum() and i < int(dataPro.shape[0]/3):
            dataPro["alphabet"][i] = 1
    return dataPro

In [5]:
onlineTest = True

if onlineTest == True:
    train =  pd.read_csv("../trainFullNumberNew88.csv",encoding = "utf-8",header = None)
    print(".............")
    l = 56
    trainx = train.iloc[:,1:l]  # 取了20维度，效果明显提升。明天试下多少合适(用循环语句)
    trainy = train.iloc[:,0]
    trainx.fillna(-888,inplace = True)
    trainx.columns = np.arange(0,l-1)
    print("trainx",trainx.head())
    xgb1 = XGBClassifier(
        #         booster = 'dart',
                learning_rate =0.1,
                n_estimators=100,
                max_depth=5,
                min_child_weight=5,
                gamma=0.1,
                subsample=0.8,
                colsample_bytree=0.8,
        #         objective= 'binary:logistic',
                nthread=3,
                scale_pos_weight=1,
                StratifiedKFold = True,          #0.866124	0.008005	0.965337	0.002286
                seed=1)
    #     xgb1.fit(trainx,trainy)
    #     prexgb = xgb1.predict(validatax)
    xgb1.fit(trainx,trainy)
    print(trainx.shape,trainy.shape)
rightNumber = 0

number = 0
fulldata = 0
kk = 0

for path,dirname,filenames in os.walk(dirs): 
    for filename in filenames:
        if filename.endswith('.csv'):  #因为有ipython 在所以必须有这句
            number += 1
            data = pd.read_csv(filename,encoding = "gb18030",header = None)
            cc = prepross(data.iloc[:,1:])
#             cc = pd.Series(cc)
            dataok = pd.concat([data.iloc[:,0],cc],axis = 1,ignore_index = True)
#             dataok.columns = ["Y","content"]
            if onlineTest == False:
                if kk == 0:
                    fulldata = dataok
                    kk = 5
                    continue
                fulldata = pd.concat([fulldata,dataok],axis = 0,ignore_index = True)
            else:
                #拼接前需要对索引重新编号
                tmp = 0
                cc.columns = np.arange(0,cc.shape[1])
#                 print(".................")
                tmp = pd.concat([trainx,cc],axis = 0, ignore_index = True)
                cc1 = tmp.iloc[trainx.shape[0]:,0:l-1]
                cc1.fillna(-888,inplace = True)
#                 print(cc1.head())
                prexgb = xgb1.predict(cc1)
                aa = accuracy_score(prexgb,data.iloc[:,0])
                if aa == 1:
                    rightNumber += 1
#                 else:
#                     print(".................")
#                     print(filename)
            if number % 10 == 0:
                print(number)
#             print(1,data.head())
#上面多了 ipython 和目录
if onlineTest == True:
    print("文件为颗粒的正确率为：",rightNumber/number)
else:
    fulldata.to_csv("../validataFullNumberNew88.csv",header = None,index = False)

.............
trainx          0   1   2   3   4   5    6      7      8      9   ...       45  \
0  0.970149   0   1   0   0   0 -666 -888.0 -888.0 -888.0  ...   -888.0   
1  0.970149   1   1   1   0   0 -888 -888.0 -888.0 -888.0  ...   -888.0   
2  0.835821   2   1   2   0   0 -888 -888.0 -888.0 -888.0  ...   -888.0   
3  0.119403   3   1   3   0   0 -666 -666.0 -666.0 -666.0  ...   -666.0   
4  0.119403   4   1   0   0   1 -666 -666.0    9.0    9.0  ...      9.0   

      46     47     48     49     50     51     52     53     54  
0 -888.0 -888.0 -888.0 -888.0 -888.0 -888.0 -888.0 -888.0 -888.0  
1 -888.0 -888.0 -888.0 -888.0 -888.0 -888.0 -888.0 -888.0 -888.0  
2 -666.0 -888.0 -888.0 -888.0 -888.0 -888.0 -666.0 -888.0 -666.0  
3 -666.0 -666.0 -666.0 -666.0 -666.0 -666.0 -666.0 -666.0 -666.0  
4    9.0    9.0    9.0    9.0    9.0    9.0    9.0    8.0    9.0  

[5 rows x 55 columns]
(56710, 55) (56710,)


d:\gongju\python3.6 64\anzhuang\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
d:\gongju\python3.6 64\anzhuang\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
d:\gongju\python3.6 64\anzhuang\lib\site-packages\ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
d:\gongju\python3.6 64\anzhuang\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set o

10


d:\gongju\python3.6 64\anzhuang\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
d:\gongju\python3.6 64\anzhuang\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
d:\gongju\python3.6 64\anzhuang\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
d:\gongju\python3.6 64\anzhuang\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambig

20


...

d:\gongju\python3.6 64\anzhuang\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
d:\gongju\python3.6 64\anzhuang\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
d:\gongju\python3.6 64\anzhuang\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
d:\gongju\python3.6 64\anzhuang\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambig

110


d:\gongju\python3.6 64\anzhuang\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
d:\gongju\python3.6 64\anzhuang\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
d:\gongju\python3.6 64\anzhuang\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
d:\gongju\python3.6 64\anzhuang\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambig

文件为颗粒的正确率为： 0.905982905982906


d:\gongju\python3.6 64\anzhuang\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [6]:
# 文件为颗粒的正确率为(15)： 0.7350427350427351    
# 文件为颗粒的正确率为(20)： 0.8034188034188035
# 文件为颗粒的正确率为(20)： 0.8034188034188035   后面都是更新后的特征
#文件为颗粒的正确率为（表格特征数目为 55）： 0.8888888888888888
#文件为颗粒的正确率为： 0.8888888888888888  所有特征


In [7]:
a = pd.DataFrame([[1,5,6],[5,6,8]],columns = ['a','1','5'])
b = pd.DataFrame([[np.nan,6,np.nan],[9,np.nan,8],[5,6,8],[np.nan,np.nan,np.nan]],columns = ['1','5','e'])
print(a,'\n',b)
# pd.concat([a,b],axis = 0,ignore_index = False)
# np.arange(0,15)
# if onlineTest = False:
#     
x = b.isnull().sum(axis = 1)

   a  1  5
0  1  5  6
1  5  6  8 
      1    5    e
0  NaN  6.0  NaN
1  9.0  NaN  8.0
2  5.0  6.0  8.0
3  NaN  NaN  NaN


In [8]:
# fulldata = pd.DataFrame(fulldata)
# label = fulldata.columns
# for i in range(fulldata.shape[0]):
#     print(fulldata[label[i].unique()])

In [9]:
# # 测试
# data = pd.read_csv("add2007nianjianhead1.csv",encoding = "gb18030",header = None)
# (data[0] == 3).sum()
# # # print("data",data.shape)
# # a = [[1,3,2],[4,6,2]]
# # (np.array(a[0]) > 1).sum()  #list很多功能都没有
# # print("2222....................")
# cc = prepross(data.iloc[:,1:])
# # cc.to_csv("add2007nianjianhead7mm.csv",encoding = "gb18030",header = None)
# cc
# # cc = pd.Series(cc)
# dataok = pd.concat([data.iloc[:,0],cc],axis = 1)
# dataok.columns = ["Y","content"]
# # len(dataok) #回来就不是  pandas

In [10]:
# mystring = u'今年18岁'
# mystring2 = mystring.encode('gbk')
# print(type(mystring2))
# print(filter(str.isdigit, mystring2))
# mm = "wo 很 开心 18"
# if mm  >= u'\u4e00' and mm <= u'\u9fa5':
#     print("包含汉字") 
# else:
#     print("bu包含汉字") 

# -*- coding: utf-8 -*-
# import re
# zhPattern = re.compile(u'[\u4e00-\u9fa5]+')
# #一个小应用，判断一段文本中是否包含简体中：
# contents=u'一个小应用，判断一段文本中是否包含简体中：'
# contents = '0.26'
# match = zhPattern.search(contents)
# if match:
#     print (u'有中文：%s' % (match.group(0),))
# else:
#     print (u'没有包含中文')